# Ansibleのセットアップ

Kafkaコネクタのセットアップを行うために [Ansible](https://docs.ansible.com/ansible/) を利用します。Ansibleを利用できるようにするための設定について簡単な説明を行います。

ここでは、以下に示す２つの設定ファイルを作成します。

* `ansible.cfg`
    - Ansibleのコンフィグレーションファイル
* `inventory.yml`
    - インベントリ
    - Ansibleの操作対象となるノードに関する設定を記述する

## `ansible.cfg` の作成

Ansibleの設定を行うには以下の方法があります（詳細については[Ansible Configuration Settings](https://docs.ansible.com/ansible/latest/reference_appendices/config.html)を参照してください）。

* `ANSIBLE_CONFIG` (環境変数)
* `ansible.cfg` (カレントディレクトリ)
* `~/.ansible.cfg` (ユーザのホームディレクトリ)
* `/etc/ansible/ansible.cfg` 

ここでは、カレントディレクトリに`ansible.cfg`を作成して設定を行います。

設定内容は以下の通りです。

* インベントリの指定
    - `inventory`
    - 次節で作成するインベントリのパスを指定します。
* コマンド実行時の警告抑止
    - `command_warnings`

カレントディレクトリの`ansible.cfg`に以下の内容を書き込みます。

```
[defaults]
inventory = $(pwd)/inventory.yml
command_warnings = False
```

> `$(pwd)`となっている箇所は実際のカレントディレクトリの値になります。

In [ ]:
cat > ansible.cfg <<EOF
[defaults]
inventory = $(pwd)/inventory.yml
command_warnings = False
EOF

cat ansible.cfg

## インベントリの作成

Ansibleを利用してノードの操作を行うためには、インベントリにノードを登録する必要があります。インベントリの作成を行います。

> ここではインベントリの書式を YAML とするので Ansible のバージョンは 2.4 以上であることを想定しています。

インベントリに記述する主な内容は以下の通りです。

* 操作対象となるノードのリスト
* 操作対象となるノードをまとめて扱うためのグループ名
* ノードにログインするためのユーザ名
* ノードにログインするためのSSHの秘密鍵
* 操作対象となるノード側での Python のパス

インベントリの例を以下に示します。

```yaml
all:
  children:
    group_name:                                      # グループ名
      hosts:
        node001:                                     # 最後にコロンを付ける
        node002:                                     # 最後にコロンを付ける
      vars:
        ansible_user: root                           # ユーザ名
        ansible_ssh_private_key_file: ~/.ssh/id_rsa  # SSHの秘密鍵
        ansible_python_interpreter: /usr/bin/python3 # Pythonインタプリタのパス
```

後で参照するので、グループ名を変数に設定しておきます。

In [ ]:
group_name=kafka_connect

インベントリを作成します。ノードのホスト名、ユーザ名などは実際の環境に合わせて書き換えてください。

In [ ]:
cat > inventory.yml <<EOF
all:
  children:
    ${group_name}:                                   # グループ名
      hosts:
        node001:                                     # 最後にコロンを付ける
      vars:
        ansible_user: root                           # ユーザ名
        ansible_ssh_private_key_file: ~/.ssh/id_rsa  # SSHの秘密鍵
        ansible_python_interpreter: /usr/bin/python3 # Pythonインタプリタのパス
EOF

cat inventory.yml

## Ansibleが利用できることを確認する

インベントリに登録したノードに対してAnsibleでアクセスできることを確認します。

次のセルを実行してください。以下のような出力結果が表示されていれば、ノードに対するアクセスが成功しています。

```
node001 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
```

次のセルを実行するには事前に `~/.ssh/known_hosts`に対象ノードが登録されていることを想定しています。登録されていない場合は、事前に

```bash
$ ssh-keyscan -H node001 >> ~/.ssh/known_hosts
```

などのコマンドを実行して `~/.ssh/known_hosts`の内容を更新してください。

In [ ]:
ansible ${group_name} -m ping

管理者権限でノードの操作が行えることを確認します。次のセルを実行してエラーにならなければ成功です。

In [ ]:
ansible ${group_name} -b -a 'whoami'